Bộ dữ liệu chạy thử nghiệm là bộ dl lấy trong bài báo Improving protein succinylation sites prediction using embeddings from protein language model
link: https://www.nature.com/articles/s41598-022-21366-2

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle as cPickle

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Bidirectional, LSTM, SimpleRNN, Input, GRU
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding, Dropout, Activation, MaxPooling1D,Conv1D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, BatchNormalization, LeakyReLU
from tensorflow.keras.initializers import glorot_uniform
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score,roc_auc_score
import sklearn.metrics as metrics
from keras import models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_train= "/content/drive/MyDrive/LBiPlantUbi/Data/"
path_test = "/content/drive/MyDrive/LBiPlantUbi/Data/"
path_result = "/content/drive/MyDrive/LBiPlantUbi/Result/"
path_model ="/content/drive/MyDrive/LBiPlantUbi/Model/"


In [ ]:
filename_train = "trainset.csv"
filename_test = "testset.csv"
file_train = filename_train
file_test = filename_test
df_train = pd.read_csv(path_train + file_train,  delimiter= ',')
df_test= pd.read_csv(path_test + filename_test,  delimiter= ',')

In [ ]:
# Def Dictionary
def  Dic_3_gram(): #Dictionary for 3_gram
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        for j in AA_list_sort:
          for jj in AA_list_sort:
             AA_dict[i+j+jj] = numm
             numm += 1
    return AA_dict

def  Dic_2_gram():#Dictionary for 2_gram
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        for j in AA_list_sort:
          AA_dict[i+j] = numm
          numm += 1
    return AA_dict

def Dic_1_gram():#Dictionary for 1_gram
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict



In [ ]:
# Separate words in sentences
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 =  Dic_1_gram()
vocab_size = len(word_index1)

In [ ]:
# preprocessing test data and index number vector
texts_train =[]
for i in df_train['Sequence']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)

data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())
print(data_token[:2])
MAX_SEQUENCE_LENGTH = len(data_token[1])

Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = df_train['Label']


[['E', 'E', 'V', 'K', 'L', 'S', 'E', 'S', 'N', 'V', 'A', 'R', 'V', 'E', 'E', 'K', 'I', 'E', 'T', 'L', 'R', 'E', 'I', 'E', 'E', 'E', 'E', 'K', 'E', 'K', 'E'], ['E', 'G', 'M', 'H', 'P', 'R', 'V', 'L', 'V', 'D', 'G', 'F', 'E', 'I', 'A', 'K', 'R', 'A', 'T', 'L', 'Q', 'F', 'L', 'D', 'N', 'F', 'K', 'T', 'P', 'V', 'V']]


In [ ]:
Xtrain[0]

array([20, 20,  3, 16,  4, 10, 20, 10, 12,  3,  2, 17,  3, 20, 20, 16,  5,
       20, 11,  4, 17, 20,  5, 20, 20, 20, 20, 16, 20, 16, 20],
      dtype=int32)

In [ ]:
# preprocessing test data and index number vector
text_test =[] #PTMsequend kmer
for i in df_test['Sequence']:
  temp = ProSentence(i,k) #
  text_test.append(temp)
df_test['k_mer'] =text_test
test_sequences = []
for each in text_test:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    test_sequences.append(each_index_list)

Xtest = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytest= df_test['Label']

In [ ]:
lr = 0.0001
num_epochs = 50
TIME_STEPS = MAX_SEQUENCE_LENGTH
INPUT_SIZE = 128 # The number of dimensions of the feature

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, Flatten, Dense,GlobalAveragePooling1D

from keras.layers import concatenate
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU, Flatten
def Classifier_hybrid(): # define classification combine LSTM_BILSTM
    model1 = Sequential()
    model1.add(Embedding(vocab_size+1, INPUT_SIZE, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model1.add(Dropout(0.5))
    model1.add(LSTM(units=64,batch_input_shape=(TIME_STEPS, INPUT_SIZE),return_sequences=True))
    model1.add(LSTM(units=64,batch_input_shape=(TIME_STEPS, INPUT_SIZE),return_sequences=True))
    model1.add(Flatten())


    model2 = Sequential()

    model2.add(Embedding(vocab_size+1, INPUT_SIZE, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model2.add(Dropout(0.5))
    model2.add(Bidirectional(LSTM(units=32,batch_input_shape=(TIME_STEPS, INPUT_SIZE),return_sequences=True)))
    model2.add(Flatten())

    merged = concatenate(([model1.output, model2.output]))
    z = Dense(128, activation="relu")(merged)
    z = Dropout(0.5)(z)

    z = Dense(1, activation="sigmoid")(z)

    model = Model(inputs=[model1.input, model2.input], outputs=z)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate = lr),
                loss='binary_crossentropy',
                metrics=['accuracy',"AUC"])


    return model

In [ ]:
model = Classifier_hybrid()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 embedding_input (InputLaye  [(None, 31)]                 0         []                            
 r)                                                                                               
                                                                                                  
 embedding (Embedding)       (None, 31, 128)              2816      ['embedding_input[0][0]']     
                                                                                                  
 embedding_1_input (InputLa  [(None, 31)]                 0         []                            
 yer)                                                                                             
                                                                                              

In [ ]:
result_hybrid = path_result + "Result_LBiPlantUbi.txt"

Test Hybrid model CNN_BiLSM

In [ ]:
# Cross CNN_BiLSTM
num_folds =10

kf = KFold(n_splits=num_folds, shuffle=True)

X = Xtrain
y = ytrain

for train_index, test_index in kf.split(X):# X1 physical feature, X2 NLP features
    X_train, X_test = X[train_index], X[test_index]

    Y_train, Y_test = y[train_index], y[test_index]
    model = Classifier_hybrid()
    model.fit([X_train,X_train], Y_train, batch_size=16, epochs=num_epochs)
    result1 = model.evaluate([X_test,X_test],Y_test)
    f = open(result_hybrid, 'a+', encoding='UTF-8')
    f.write("\n Crossvalidation test 5.7.2024")

    s = str(result1)
    f.write(s)
    f.close()
    # Define your model
    # Compile the model with the desired optimizer, loss function, and metrics

    y_pred= model.predict([X_test,X_test])
    ytest_true = Y_test
    #y_pred = model.predict([Xtest,Xtest])

    y_pred = (y_pred > 0.5)

    y_pred = [int(i) for i in y_pred]

    y_pred = np.array(y_pred)
    y_true = np.array(ytest_true)

    cm = confusion_matrix(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)

    sn = cm[1][1]/(cm[1][1]+cm[1][0])
    sp = cm[0][0]/(cm[0][0]+cm[0][1])
    print(acc, mcc,sn,sp)
    print(cm)

    f = open(result_hybrid, 'a+', encoding='UTF-8')
    f.write("\n crossvalidation confusion_matric\n ")
    s = str(cm)
    f.write(s)
    f.close()

Epoch 1/50
310/310 [==============================] - 20s 34ms/step - loss: 0.6377 - accuracy: 0.6566 - auc: 0.7089
Epoch 2/50
310/310 [==============================] - 11s 34ms/step - loss: 0.5724 - accuracy: 0.7149 - auc: 0.7756
Epoch 3/50
310/310 [==============================] - 10s 34ms/step - loss: 0.5661 - accuracy: 0.7214 - auc: 0.7813
Epoch 4/50
310/310 [==============================] - 10s 34ms/step - loss: 0.5575 - accuracy: 0.7305 - auc: 0.7895
Epoch 5/50
310/310 [==============================] - 10s 33ms/step - loss: 0.5460 - accuracy: 0.7354 - auc: 0.8003
Epoch 6/50
310/310 [==============================] - 11s 34ms/step - loss: 0.5402 - accuracy: 0.7360 - auc: 0.8052
Epoch 7/50
310/310 [==============================] - 10s 33ms/step - loss: 0.5290 - accuracy: 0.7469 - auc: 0.8149
Epoch 8/50
310/310 [==============================] - 10s 33ms/step - loss: 0.5260 - accuracy: 0.7438 - auc: 0.8172
Epoch 9/50
310/310 [==============================] - 10s 33ms/step - lo

<ipython-input-72-07de00e87821>:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred = [int(i) for i in y_pred]


Epoch 1/50
310/310 [==============================] - 19s 33ms/step - loss: 0.6372 - accuracy: 0.6503 - auc: 0.7027
Epoch 2/50
310/310 [==============================] - 10s 33ms/step - loss: 0.5783 - accuracy: 0.7113 - auc: 0.7691
Epoch 3/50
310/310 [==============================] - 10s 33ms/step - loss: 0.5682 - accuracy: 0.7236 - auc: 0.7791
Epoch 4/50
310/310 [==============================] - 10s 33ms/step - loss: 0.5617 - accuracy: 0.7230 - auc: 0.7853
Epoch 5/50
310/310 [==============================] - 10s 32ms/step - loss: 0.5549 - accuracy: 0.7301 - auc: 0.7919
Epoch 6/50
310/310 [==============================] - 10s 32ms/step - loss: 0.5433 - accuracy: 0.7370 - auc: 0.8021
Epoch 7/50
310/310 [==============================] - 10s 33ms/step - loss: 0.5372 - accuracy: 0.7426 - auc: 0.8083
Epoch 8/50
310/310 [==============================] - 10s 32ms/step - loss: 0.5295 - accuracy: 0.7471 - auc: 0.8137
Epoch 9/50
310/310 [==============================] - 10s 32ms/step - lo

<ipython-input-72-07de00e87821>:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred = [int(i) for i in y_pred]


Epoch 1/50
310/310 [==============================] - 17s 32ms/step - loss: 0.6333 - accuracy: 0.6570 - auc: 0.7080
Epoch 2/50
310/310 [==============================] - 10s 32ms/step - loss: 0.5711 - accuracy: 0.7222 - auc: 0.7768
Epoch 3/50
310/310 [==============================] - 10s 32ms/step - loss: 0.5612 - accuracy: 0.7220 - auc: 0.7866
Epoch 4/50
310/310 [==============================] - 10s 32ms/step - loss: 0.5547 - accuracy: 0.7321 - auc: 0.7929
Epoch 5/50
310/310 [==============================] - 10s 33ms/step - loss: 0.5499 - accuracy: 0.7293 - auc: 0.7969
Epoch 6/50
310/310 [==============================] - 10s 33ms/step - loss: 0.5435 - accuracy: 0.7396 - auc: 0.8026
Epoch 7/50
310/310 [==============================] - 10s 32ms/step - loss: 0.5375 - accuracy: 0.7347 - auc: 0.8070
Epoch 8/50
310/310 [==============================] - 10s 32ms/step - loss: 0.5283 - accuracy: 0.7451 - auc: 0.8148
Epoch 9/50
310/310 [==============================] - 10s 33ms/step - lo

...

<ipython-input-72-07de00e87821>:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred = [int(i) for i in y_pred]


Epoch 1/50
310/310 [==============================] - 17s 32ms/step - loss: 0.6330 - accuracy: 0.6461 - auc: 0.7030
Epoch 2/50
310/310 [==============================] - 10s 31ms/step - loss: 0.5723 - accuracy: 0.7198 - auc: 0.7751
Epoch 3/50
310/310 [==============================] - 10s 31ms/step - loss: 0.5617 - accuracy: 0.7273 - auc: 0.7861
Epoch 4/50
310/310 [==============================] - 10s 32ms/step - loss: 0.5483 - accuracy: 0.7374 - auc: 0.7984
Epoch 5/50
310/310 [==============================] - 10s 32ms/step - loss: 0.5439 - accuracy: 0.7360 - auc: 0.8034
Epoch 6/50
310/310 [==============================] - 10s 32ms/step - loss: 0.5356 - accuracy: 0.7440 - auc: 0.8111
Epoch 7/50
310/310 [==============================] - 10s 31ms/step - loss: 0.5266 - accuracy: 0.7527 - auc: 0.8175
Epoch 8/50
310/310 [==============================] - 10s 32ms/step - loss: 0.5179 - accuracy: 0.7533 - auc: 0.8234
Epoch 9/50
310/310 [==============================] - 10s 32ms/step - lo

<ipython-input-72-07de00e87821>:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred = [int(i) for i in y_pred]


Epoch 1/50
310/310 [==============================] - 20s 34ms/step - loss: 0.6351 - accuracy: 0.6390 - auc: 0.6982
Epoch 2/50
310/310 [==============================] - 10s 33ms/step - loss: 0.5754 - accuracy: 0.7129 - auc: 0.7729
Epoch 3/50
310/310 [==============================] - 12s 39ms/step - loss: 0.5631 - accuracy: 0.7242 - auc: 0.7859
Epoch 4/50
310/310 [==============================] - 11s 34ms/step - loss: 0.5538 - accuracy: 0.7281 - auc: 0.7946
Epoch 5/50
310/310 [==============================] - 11s 34ms/step - loss: 0.5438 - accuracy: 0.7392 - auc: 0.8025
Epoch 6/50
310/310 [==============================] - 11s 36ms/step - loss: 0.5371 - accuracy: 0.7356 - auc: 0.8078
Epoch 7/50
310/310 [==============================] - 12s 38ms/step - loss: 0.5305 - accuracy: 0.7463 - auc: 0.8140
Epoch 8/50
310/310 [==============================] - 12s 39ms/step - loss: 0.5228 - accuracy: 0.7475 - auc: 0.8210
Epoch 9/50
310/310 [==============================] - 11s 34ms/step - lo

<ipython-input-72-07de00e87821>:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred = [int(i) for i in y_pred]
